In [1]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

/home/max/micromamba/envs/pol_stats/lib/python3.12/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "cannot import name 'Callable' from 'collections' (/home/max/micromamba/envs/pol_stats/lib/python3.12/collections/__init__.py)".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

$\dot{L}=gL$ 

$\dot{g}=-\alpha(g-\beta \frac{L_{end}-L}{L_{end}})$ if $L<L_{cut}$

Combine Dev and Reg with differnt initial conditions

In [2]:
rng = np.random.default_rng()

from scipy.integrate import solve_ivp

def L_theor(t, L_0, g_0, alpha, beta, L_end,L_cut):
    t48 = t - 48

    def ode_system(t, y):
        L, g = y
        dLdt = g*L
        if L < L_cut:
            dgdt = -alpha * (g - beta * (L_end - L_cut) / L_end)
        else:
            dgdt = -alpha * (g - beta * (L_end - L) / L_end)
        return [dLdt, dgdt]

    # Initial conditions
    y0 = [L_0, g_0]

    # Solve the ODE
    solution = solve_ivp(ode_system, [t48.min(), t48.max()], y0, t_eval=t48, method='RK45')
    #print("Solver message:", solution.message)
    return solution.y[0]

def linCrit_ppc(x):
    sigma = 0*np.abs(rng.normal(0, 2.0))
    L_0 = 10**rng.normal(0.3, 0.15)
    g_0 = np.abs(rng.normal(0, 0.1))
    alpha = 10**rng.normal(-0.25, 0.25)
    beta = (alpha/4)*10**rng.normal(0, 0.1)
    L_end = 10**rng.normal(1.0, 0.1)
    L_cut = 2 + 4*rng.beta(5.0,5.0)
    #print(L_0, g_0,alpha, beta, L_end)
    return (rng.normal(L_theor(x, L_0, g_0,alpha, beta, L_end,L_cut), sigma)/L_end,
            {"L_0": L_0, "g_0": g_0, "alpha": alpha, "beta": beta, "L_end": L_end, "L_cut":L_cut})

In [3]:
x = np.linspace(48, 144, 30)
for _ in range(10):
    print(linCrit_ppc(x))

(array([0.18295573, 0.27219022, 0.32705513, 0.37820289, 0.43437342,
       0.49826693, 0.57144183, 0.65528585, 0.74449379, 0.82306699,
       0.88398017, 0.92719552, 0.95587099, 0.97393305, 0.98487279,
       0.99133928, 0.99508511, 0.99722734, 0.99844034, 0.99913501,
       0.9995205 , 0.99971974, 0.99983686, 0.9999095 , 0.99994853,
       0.99997124, 0.99998355, 0.99999059, 0.99999549, 0.99999736]), {'L_0': 1.4345908448203137, 'g_0': np.float64(0.20605286183447427), 'alpha': 0.5196870754883057, 'beta': 0.11642155601003559, 'L_end': 7.84119129936636, 'L_cut': 5.0544640279136335})
(array([0.28376708, 0.46317547, 0.65085583, 0.81162564, 0.91721016,
       0.97054654, 0.99177759, 0.99849529, 1.00007238, 1.00022201,
       1.00012758, 1.00005225, 1.00001718, 1.00000386, 1.00000039,
       0.99999959, 0.99999944, 1.00000029, 1.00000141, 1.00000289,
       0.99999985, 0.99999658, 0.99999701, 0.99999974, 1.00000144,
       1.0000011 , 0.99999931, 0.9999994 , 0.99999982, 1.00000039]), {'L_0':

In [4]:
x = np.linspace(48, 144, 100)
f_ppc = [linCrit_ppc(x) for _ in range(100)]

p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

for f, params in f_ppc:
    line = p.line(x, f)
    p.add_tools(bokeh.models.HoverTool(
        renderers=[line],
        tooltips=[(key, f"{params[key]}") for key in params.keys()]
    ))

bokeh.io.show(p)

In [5]:
df = pd.read_csv('../../data/area_df.csv')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="total_area-6",
    #x_axis_type="log",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_3
items = []
for color, (genotype, sub_df) in zip(colors, df.groupby("category")):
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["total_area-6"],
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)

In [6]:
sm = cmdstanpy.CmdStanModel(stan_file='gLin_combined.stan')

15:22:56 - cmdstanpy - INFO - compiling stan file /home/max/Documents/01_Code/BaysianGrowth/code/gLin/gLin_combined.stan to exe file /home/max/Documents/01_Code/BaysianGrowth/code/gLin/gLin_combined
15:23:04 - cmdstanpy - INFO - compiled model executable: /home/max/Documents/01_Code/BaysianGrowth/code/gLin/gLin_combined


In [7]:
def df_to_data(cat_Dev,cat_Reg):
    filtered_df = df[df['category'] == cat_Reg]
    t_Reg = filtered_df['time in hpf'].to_numpy()
    L_Reg = filtered_df['total_area-6'].to_numpy()
    sorted_indices = np.argsort(t_Reg)
    t_Reg = t_Reg[sorted_indices]
    L_Reg = L_Reg[sorted_indices]

    filtered_df = df[df['category'] == cat_Dev]
    t_Dev = filtered_df['time in hpf'].to_numpy()
    L_Dev = filtered_df['total_area-6'].to_numpy()
    sorted_indices = np.argsort(t_Dev)
    t_Dev = t_Dev[sorted_indices]
    L_Dev = L_Dev[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(48, 144, N_ppc)
    
    return dict(t_Reg=t_Reg, L_Reg=L_Reg, N_Reg=len(t_Reg),
                t_Dev=t_Dev, L_Dev=L_Dev, N_Dev=len(t_Dev),
                  N_ppc=N_ppc, t_ppc=t_ppc)

data = df_to_data('Development','Regeneration')

In [8]:
samples = sm.sample(data=data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['L_Dev_ppc','L_Reg_ppc'])
bebi103.stan.check_all_diagnostics(samples)

15:23:04 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

15:24:23 - cmdstanpy - INFO - CmdStan done processing.
15:24:23 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: ode_rk45: initial state[1] is inf, but must be finite! (in 'gLin_combined.stan', line 28, column 8 to column 91) (in 'gLin_combined.stan', line 97, column 4 to column 88)
	Exception: Exception: ode_rk45: ode parameters and data[3] is inf, but must be finite! (in 'gLin_combined.stan', line 28, column 8 to column 91) (in 'gLin_combined.stan', line 97, column 4 to column 88)
Exception: Exception: ode_rk45: ode parameters and data[1] is inf, but must be finite! (in 'gLin_combined.stan', line 28, column 8 to column 91) (in 'gLin_combined.stan', line 97, column 4 to column 88)
	Exception: Exception: ode_rk45: ode parameters and data[1] is inf, but must be finite! (in 'gLin_combined.stan', line 28, column 8 to column 91) (in 'gLin_combined.stan', line 97, column 4 to column 88)
	Exception: Exception: ode_rk45: ode parameters and data[1] is inf, but mus


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

0 of 4000 (0.0%) iterations ended with a divergence.

0 of 4000 (0.0%) iterations saturated the maximum tree depth of 10.

E-BFMI indicated no pathological behavior.


0

In [9]:
output_file("plots/gLinCut_DevREG_ppc_corner.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['L_end','L_cut','g_0_Dev','g_0_Reg', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [10]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [11]:
output_file("plots/gLinCut_DevREG_ppc_Lt_Dev.html")
f_ppc = samples.posterior_predictive.L_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "L_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Dev"], data["L_Dev"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='L',
    )
)

In [12]:
output_file("plots/gLinCut_DevREG_ppc_Lt_Reg.html")
f_ppc = samples.posterior_predictive.L_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "L_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Reg"], data["L_Reg"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='L',
    )
)

In [13]:
bebi103.stan.clean_cmdstan()